# Testing the Expense Fetching Function

In [3]:
import imaplib
import email
from datetime import datetime, timedelta

# Email Server
imap_server = "imap.gmail.com"
email_address = "rybott5000.5@gmail.com"
pw = "piau rbar zvoy zirl"
imap = imaplib.IMAP4_SSL(imap_server)
imap.login(email_address, pw)

# Start End Date
end_date = datetime.now()
start_date = end_date - timedelta(days=730)
start_str = start_date.strftime("%d-%b-%Y")
end_str = (end_date + timedelta(days=1)).strftime("%d-%b-%Y")

# Search Email
imap.select("Transactions")
_, msgnums = imap.search(None, f'(SINCE "{start_str}" BEFORE "{end_str}")')

contents = []

for msgnum in msgnums[0].split():
  for _ in range(1):
    _, data = imap.fetch(msgnum, "(RFC822)")
    message = email.message_from_bytes(data[0][1])
    for part in message.walk():
      if part.get_content_type() == "text/plain":
        contents.append(part.as_string())


imap.close()

('OK', [b'Returned to authenticated state. (Success)'])

In [4]:
import re
import pandas as pd

Sender = []
Merchant = []
Amount = []
Date = []


for email_content in contents:

  # Sender
  if 'Discover' in email_content:
    sender = 'Discover'
  elif 'Chase' in email_content:
    sender = 'Chase'
  else:
    sender = 'Other'
  Sender.append(sender)
  
  # Pattern for Merchant
  merchant_pattern = r"Merchant:\s*(?P<merchant>.*?)\n"
  merchant_match = re.search(merchant_pattern, email_content)
  if merchant_match:
      merchant = merchant_match.group("merchant")
  else:
      merchant = None
  Merchant.append(merchant)

  # Pattern for Amount
  amount_pattern = r"Amount:\s*\$(?P<amount>[\d\.]+)"
  amount_match = re.search(amount_pattern, email_content)
  if amount_match:
      amount = amount_match.group("amount")
  else:
      amount = None
  Amount.append(amount)

  # Pattern for Transaction Date
  date_pattern = r"Transaction Date::\s*(?P<date>.*?)\n"
  date_match = re.search(date_pattern, email_content)
  if date_match:
      transaction_date = date_match.group("date")
  else:
      transaction_date = None
  Date.append(transaction_date)

Transactions_df = pd.DataFrame({
  'Sender':Sender,
  'Transaction_Date':Date,
  'Merchant':Merchant,
  'Amount':Amount
})

print(Transactions_df)

       Sender    Transaction_Date                Merchant Amount
0    Discover  September 27, 2023     PAYPAL *DISNEY PLUS   7.99
1    Discover  September 27, 2023          MTA*NYCT PAYGO   2.90
2    Discover    October 12, 2023          MTA*NYCT PAYGO   2.90
3    Discover    October 12, 2023      DUNKIN #351785 Q35   5.65
4    Discover    October 12, 2023          MTA*NYCT PAYGO   2.90
..        ...                 ...                     ...    ...
622  Discover       July 22, 2024    AMAZON MKTPLACE PMTS  71.61
623  Discover       July 22, 2024  GOOGLE *YouTubePremium   7.99
624  Discover       July 26, 2024     PAYPAL *STARBUCKSSE   7.35
625  Discover       July 28, 2024        Prime Video TVOD   3.79
626  Discover       July 28, 2024    AMAZON MKTPLACE PMTS  11.39

[627 rows x 4 columns]


In [ ]:
with pd.ExcelWriter('Discover Transaction.xlsx') as writer:
    Transactions_df.to_excel(writer, sheet_name='Sheet1', index=False)